# Getting the Dataset
Download the files if you would like to reproduce my training

If you have the files, just replace the path with the path to your local install

I will fix the seed for numpy and torch if you'd like to verify the results

In [1]:
# UNCOMMENT AND RUN IF RUNNING IN COLAB

# Single Photon .hdf5
!wget -O single_photon.hdf5 https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5

# Single Electron .hdf5
!wget -O single_electron.hdf5  https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5

--2025-04-03 06:59:15--  https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.53.35, 137.138.120.151, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.53.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119703858 (114M) [application/octet-stream]
Saving to: ‘single_photon.hdf5’

single_photon.hdf5  100%[===================>] 114.16M   531KB/s    in 37s     

2025-04-03 06:59:54 (3.07 MB/s) - ‘single_photon.hdf5’ saved [119703858/119703858]

--2025-04-03 06:59:54--  https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.53.35, 137.138.120.151, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.53.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128927319 (123M) [

In [2]:
single_photon_path = 'single_photon.hdf5' # Replace with the path to your single photon .hdf5 file
single_electron_path = 'single_electron.hdf5' # Replace with the path to your single electron .hdf5 file

# Reading the Data and preparing the Dataset

In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os


In [4]:
with h5py.File(single_photon_path, 'r') as f:
    # Print all root level object names (aka keys)
    print("Keys in single photon file:")
    for key in f.keys():
        print(key)
    X_photon = f['X'][:]
    y_photon = f['y'][:]
    print("Shape of X_photon:", X_photon.shape)
    print("Shape of y_photon:", y_photon.shape)


Keys in single photon file:
X
y
Shape of X_photon: (249000, 32, 32, 2)
Shape of y_photon: (249000,)


In [5]:
with h5py.File(single_electron_path, 'r') as f:
    # Print all root level object names (aka keys)
    print("Keys in single electron file:")
    for key in f.keys():
        print(key)
    X_electron = f['X'][:]
    y_electron = f['y'][:]
    print("Shape of X_electron:", X_electron.shape)
    print("Shape of y_electron:", y_electron.shape)

Keys in single electron file:
X
y
Shape of X_electron: (249000, 32, 32, 2)
Shape of y_electron: (249000,)


In [6]:
# Get the value of y_photon
unique_y_photon = np.unique(y_photon)
print("Unique y_photon values:", unique_y_photon)

# Get the value of y_electron
unique_y_electron = np.unique(y_electron)
print("Unique y_electron values:", unique_y_electron)


Unique y_photon values: [0.]
Unique y_electron values: [1.]


Okay so a binary classification task

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

X = torch.Tensor(np.concatenate((X_photon, X_electron), axis=0))
X = X.permute(0, 3, 1, 2) # Change the order of dimensions to (N, C, H, W)
y = torch.Tensor(np.concatenate((y_photon, y_electron), axis=0))

dataset = TensorDataset(X, y)
print(f"Dataset: {dataset}")


Dataset: <torch.utils.data.dataset.TensorDataset object at 0x7a9ab8d8ac90>


In [13]:
train_data,val_data = torch.utils.data.random_split(dataset,[int(len(dataset)*0.8),int(len(dataset)*0.2)])

# Create a DataLoader
batch_size = 512
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=2)

# Check the first batch of data
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)
    print("Batch y shape:", batch_y.shape)
    break  # Just show the first batch

Batch X shape: torch.Size([512, 2, 32, 32])
Batch y shape: torch.Size([512])


# Defining ResNet 15 Model Architecture

In [16]:
import torchvision.models as models
import torch.nn as nn

# Resnet18 model
model = models.resnet18(weights='IMAGENET1K_V1')
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
#Modify conv1 to accept 1x32x32 input
model.conv1 = nn.Conv2d(in_channels=2, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
nn.init.kaiming_uniform_(model.conv1.weight, nonlinearity='relu')

# Add additional fully connected layers for binary classification
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)
# randomly initialize the weights of the new layers
for layer in model.fc:
    if isinstance(layer, nn.Linear):
        nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)

print(model)
print(model.conv1.weight.shape)

ResNet(
  (conv1): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# Test forward pass with a single photon image
test_image = torch.tensor(X_photon[0], dtype=torch.float32).unsqueeze(0)  # Add batch dimension
test_image = test_image.permute(0, 3, 1, 2)  # Change to (batch_size, channels, height, width)
print("Test image shape:", test_image.shape)
model.eval()
output = model(test_image)
print("Output shape:", output.shape)
print("Output:", output)

print(f"Class is {torch.argmax(output, dim=1).item()}")

Test image shape: torch.Size([1, 2, 32, 32])
Output shape: torch.Size([1, 2])
Output: tensor([[0.4059, 0.5941]], grad_fn=<SoftmaxBackward0>)
Class is 1


# Setting the Training loop

In [18]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.nn as nn
import torch

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
num_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader):
        batch_X, batch_y = batch
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.long())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch_X, batch_y = batch
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.long())
            val_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")


  0%|          | 0/779 [00:00<?, ?it/s]

Epoch [1/25], Train Loss: 0.6713, Val Loss: 0.6392, Val Acc: 0.6408


  0%|          | 0/779 [00:00<?, ?it/s]

Epoch [2/25], Train Loss: 0.6155, Val Loss: 0.6454, Val Acc: 0.6305


  0%|          | 0/779 [00:00<?, ?it/s]

...

Epoch [23/25], Train Loss: 0.5231, Val Loss: 0.5574, Val Acc: 0.7232


  0%|          | 0/779 [00:00<?, ?it/s]

Epoch [24/25], Train Loss: 0.5228, Val Loss: 0.5564, Val Acc: 0.7235


  0%|          | 0/779 [00:00<?, ?it/s]

Epoch [25/25], Train Loss: 0.5227, Val Loss: 0.5576, Val Acc: 0.7228
